#### Import necessary dependensies

In [ ]:
import pandas as pd
import geopandas as gpd
import folium

#### Load the CSV file with ward uptake rate

In [ ]:
df_csv = pd.read_csv("https://2207-17-fibre-competitive-intensity-model-b.s3.eu-west-1.amazonaws.com/SA-uptake-rate-ward-level.csv")

#### Display the first few rows of the DataFrame

In [ ]:
df_csv.head()

#### Load the shapefile of South Africa wards

In [ ]:
# Uncheck the url to the S3 bucket and check the path to the local pc

# gdf_ward = gpd.read_file("https://2207-17-fibre-competitive-intensity-model-b.s3.eu-west-1.amazonaws.com/2011+Wards+Administrative+boundaries/Wards2011.shp")

gdf_ward = gpd.read_file("C:/Users/nmwem/OneDrive/Desktop/SA-Maps/Wards/Wards2011.shp")

#### Display the first few rows of the DataFrame

In [ ]:
gdf_ward.head()

#### Rename the 'WARD_ID' column

In [ ]:
gdf_ward = gdf_ward.rename(columns={
    'WARD_ID': 'ward_code'
})

#### Display the first few columns of the df

In [ ]:
gdf_ward.head()

#### Check the info of the df to determine if the dtypes are compatible for merging

In [ ]:
df_csv.info()

#### Check the info of the df to determine if the dtypes are compatible for merging

In [ ]:
gdf_ward.info()

#### Convert ward_code dtype in gdf_ward to numeric

In [ ]:
gdf_ward['ward_code'] = pd.to_numeric(gdf_ward['ward_code'])

#### Merge the two DataFrames based on the municipality name

In [ ]:
merged_df = pd.merge(gdf_ward, df_csv, on="ward_code")

#### Display the first few rows of the DataFrame

In [ ]:
merged_df.head()

In [ ]:
merged_df.info()

#### Convert the merged DataFrame into a GeoJSON file

In [ ]:
merged_geojson = merged_df.to_crs(epsg='4326').to_json()

#### Create the Folium map

In [ ]:
m = folium.Map(location=[-29.0562, 25.4377], zoom_start=5)

#### Add the choropleth layer to the map

In [ ]:
folium.Choropleth(
    geo_data=merged_geojson,
    data=merged_df,
    columns=['ward_code', 'uptake_rate_pop'],
    key_on='feature.properties.ward_code',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Fibre Uptake Rate_pop (%)'
).add_to(m)

#### Add GeoJsonTooltip to show the exact uptake rate

In [ ]:
folium.GeoJson(
    merged_geojson,
    name='tooltip',
    tooltip=folium.features.GeoJsonTooltip(
        fields=['MUNICNAME_x', 'ward_code', 'uptake_rate_pop'],
        aliases=['Municipality:', 'Ward:', 'Uptake Rate:'],
        style=('background-color: grey; color: white;')
    )
).add_to(m)

#### Display the map

In [ ]:
m